In [34]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import lit, mean, col
import os
import configparser
from pandas import set_option
set_option('display.max_columns', None)
set_option('display.max_rows', None)


In [2]:
config = configparser.RawConfigParser()
path = os.path.join(os.path.expanduser('~'), '.aws/credentials')
config.read(path)

['/home/gustavoalmeida/.aws/credentials']

In [3]:
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4')
conf.set("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com")
conf.set('spark.hadoop.fs.s3a.access.key', config.get('datasouls', 'aws_access_key_id'))
conf.set('spark.hadoop.fs.s3a.secret.key', config.get('datasouls', 'aws_secret_access_key'))
conf.set('spark.executor.memory', '8g')
conf.set('spark.driver.memory', '8g')
conf.set('spark.executor.cores', '4')
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [4]:
spark = SparkSession.builder.appName("eda").config(conf=conf).getOrCreate()

22/08/11 22:25:04 WARN Utils: Your hostname, gustavoalmeida resolves to a loopback address: 127.0.1.1; using 172.27.160.28 instead (on interface eth0)
22/08/11 22:25:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/gustavoalmeida/.ivy2/cache
The jars for the packages stored in: /home/gustavoalmeida/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9cf9aa68-5bee-4e0c-82e9-01493a95b325;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 94ms :: artifacts dl 3ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	------------------

22/08/11 22:25:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
schema_microdados = StructType([
    StructField("NU_INSCRICAO", IntegerType(), True),
    StructField("NU_ANO", IntegerType(), True),
    StructField("TP_FAIXA_ETARIA", IntegerType(), True),
    StructField("TP_SEXO", StringType(), True),
    StructField("TP_ESTADO_CIVIL", IntegerType(), True),
    StructField("TP_COR_RACA", IntegerType(), True),
    StructField("TP_NACIONALIDADE", IntegerType(), True),
    StructField("TP_ST_CONCLUSAO", IntegerType(), True),
    StructField("TP_ANO_CONCLUIU", IntegerType(), True),
    StructField("TP_ESCOLA", IntegerType(), True),
    StructField("TP_ENSINO", IntegerType(), True),
    StructField("IN_TREINEIRO", IntegerType(), True),
    StructField("CO_MUNICIPIO_ESC", IntegerType(), True),
    StructField("NO_MUNICIPIO_ESC", StringType(), True),
    StructField("CO_UF_ESC", IntegerType(), True),
    StructField("SG_UF_ESC", StringType(), True),
    StructField("TP_DEPENDENCIA_ADM_ESC", IntegerType(), True),
    StructField("TP_LOCALIZACAO_ESC", IntegerType(), True),
    StructField("TP_SIT_FUNC_ESC", IntegerType(), True),
    StructField("CO_MUNICIPIO_PROVA", IntegerType(), True),
    StructField("NO_MUNICIPIO_PROVA", StringType(), True),
    StructField("CO_UF_PROVA", StringType(), True),
    StructField("SG_UF_PROVA", StringType(), True),
    StructField("TP_PRESENCA_CN", IntegerType(), True),
    StructField("TP_PRESENCA_CH", IntegerType(), True),
    StructField("TP_PRESENCA_LC", IntegerType(), True),
    StructField("TP_PRESENCA_MT", IntegerType(), True),
    StructField("CO_PROVA_CN", IntegerType(), True),
    StructField("CO_PROVA_CH", IntegerType(), True),
    StructField("CO_PROVA_LC", IntegerType(), True),
    StructField("CO_PROVA_MT", IntegerType(), True),
    StructField("NU_NOTA_CN", IntegerType(), True),
    StructField("NU_NOTA_CH", IntegerType(), True),
    StructField("NU_NOTA_LC", IntegerType(), True),
    StructField("NU_NOTA_MT", IntegerType(), True),
    StructField("TX_RESPOSTAS_CN", StringType(), True),
    StructField("TX_RESPOSTAS_CH", StringType(), True),
    StructField("TX_RESPOSTAS_LC", StringType(), True),
    StructField("TX_RESPOSTAS_MT", StringType(), True),
    StructField("TP_LINGUA", IntegerType(), True),
    StructField("TX_GABARITO_CN", StringType(), True),
    StructField("TX_GABARITO_CH", StringType(), True),
    StructField("TX_GABARITO_LC", StringType(), True),
    StructField("TX_GABARITO_MT", StringType(), True),
    StructField("TP_STATUS_REDACAO", IntegerType(), True),
    StructField("NU_NOTA_COMP1", IntegerType(), True),
    StructField("NU_NOTA_COMP2", IntegerType(), True),
    StructField("NU_NOTA_COMP3", IntegerType(), True),
    StructField("NU_NOTA_COMP4", IntegerType(), True),
    StructField("NU_NOTA_COMP5", IntegerType(), True),
    StructField("NU_NOTA_REDACAO", IntegerType(), True),
    StructField("Q001", StringType(), True),
    StructField("Q002", StringType(), True),
    StructField("Q003", StringType(), True),
    StructField("Q004", StringType(), True),
    StructField("Q005", StringType(), True),
    StructField("Q006", StringType(), True),
    StructField("Q007", StringType(), True),
    StructField("Q008", StringType(), True),
    StructField("Q009", StringType(), True),
    StructField("Q010", StringType(), True),
    StructField("Q011", StringType(), True),
    StructField("Q012", StringType(), True),
    StructField("Q013", StringType(), True),
    StructField("Q014", StringType(), True),
    StructField("Q015", StringType(), True),
    StructField("Q016", StringType(), True),
    StructField("Q017", StringType(), True),
    StructField("Q018", StringType(), True),
    StructField("Q019", StringType(), True),
    StructField("Q020", StringType(), True),
    StructField("Q021", StringType(), True),
    StructField("Q022", StringType(), True),
    StructField("Q023", StringType(), True),
    StructField("Q024", StringType(), True),
    StructField("Q025", StringType(), True), 
])

In [6]:
microdados_bronze_read_path = 's3a://enem-pyspark-athena-bronze/2020/microdados/MICRODADOS_ENEM_2020.csv'
microdados_silver_write_path = 's3a://enem-pyspark-athena-silver/2020/microdados/'

In [7]:
df_microdados = spark.read \
        .options(delimiter=';', 
                 header='True',
                 encoding='latin1') \
        .schema(schema_microdados) \
        .csv(microdados_bronze_read_path)

22/08/11 22:25:19 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [18]:
df_microdados.show(vertical=True)

-RECORD 0--------------------------------------
 NU_INSCRICAO           | null                 
 NU_ANO                 | 2020                 
 TP_FAIXA_ETARIA        | 11                   
 TP_SEXO                | F                    
 TP_ESTADO_CIVIL        | 1                    
 TP_COR_RACA            | 2                    
 TP_NACIONALIDADE       | 1                    
 TP_ST_CONCLUSAO        | 1                    
 TP_ANO_CONCLUIU        | 11                   
 TP_ESCOLA              | 1                    
 TP_ENSINO              | null                 
 IN_TREINEIRO           | 0                    
 CO_MUNICIPIO_ESC       | null                 
 NO_MUNICIPIO_ESC       | null                 
 CO_UF_ESC              | null                 
 SG_UF_ESC              | null                 
 TP_DEPENDENCIA_ADM_ESC | null                 
 TP_LOCALIZACAO_ESC     | null                 
 TP_SIT_FUNC_ESC        | null                 
 CO_MUNICIPIO_PROVA     | 1501402       

In [9]:
df_microdados.printSchema()

root
 |-- NU_INSCRICAO: integer (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: strin

In [10]:
schema_itens_prova = StructType([
    StructField("CO_POSICAO", IntegerType(), True),
    StructField("SG_AREA", StringType(), True),
    StructField("CO_ITEM", IntegerType(), True),
    StructField("TX_GABARITO", StringType(), True),
    StructField("CO_HABILIDADE", IntegerType(), True),
    StructField("IN_ITEM_ABAN", IntegerType(), True),
    StructField("TX_MOTIVO_ABAN", StringType(), True),
    StructField("NU_PARAM_A", DoubleType(), True),
    StructField("NU_PARAM_B", DoubleType(), True),
    StructField("NU_PARAM_C", DoubleType(), True),
    StructField("TX_COR", StringType(), True),
    StructField("CO_PROVA", IntegerType(), True),
    StructField("TP_LINGUA", IntegerType(), True),
    StructField("IN_ITEM_ADAPTADO", IntegerType(), True),
    StructField("TP_VERSAO_DIGITAL", IntegerType(), True),
])

In [11]:
itens_prova_bronze_read_path = 's3a://enem-pyspark-athena-bronze/2020/itens_prova/ITENS_PROVA_2020.csv'
itens_prova_silver_write_path = 's3a://enem-pyspark-athena-silver/2020/itens_prova/'

In [12]:
df_itens_prova = spark.read \
        .options(delimiter=';', 
                 header='True',
                 encoding='latin1') \
        .schema(schema_itens_prova) \
        .csv(itens_prova_bronze_read_path)

In [13]:
df_itens_prova.show()

+----------+-------+-------+-----------+-------------+------------+--------------+----------+----------+----------+----------------+--------+---------+----------------+-----------------+
|CO_POSICAO|SG_AREA|CO_ITEM|TX_GABARITO|CO_HABILIDADE|IN_ITEM_ABAN|TX_MOTIVO_ABAN|NU_PARAM_A|NU_PARAM_B|NU_PARAM_C|          TX_COR|CO_PROVA|TP_LINGUA|IN_ITEM_ADAPTADO|TP_VERSAO_DIGITAL|
+----------+-------+-------+-----------+-------------+------------+--------------+----------+----------+----------+----------------+--------+---------+----------------+-----------------+
|        26|     LC|   7099|          D|           15|           0|          null|   1.33337|  -0.37357|   0.00658|LARANJA - BRAILE|     623|     null|               1|             null|
|        43|     LC|   7099|          D|           15|           0|          null|   1.33337|  -0.37357|   0.00658|          BRANCA|     620|     null|               0|             null|
|        26|     LC|   7099|          D|           15|           

In [14]:
df_itens_prova.printSchema()

root
 |-- CO_POSICAO: integer (nullable = true)
 |-- SG_AREA: string (nullable = true)
 |-- CO_ITEM: integer (nullable = true)
 |-- TX_GABARITO: string (nullable = true)
 |-- CO_HABILIDADE: integer (nullable = true)
 |-- IN_ITEM_ABAN: integer (nullable = true)
 |-- TX_MOTIVO_ABAN: string (nullable = true)
 |-- NU_PARAM_A: double (nullable = true)
 |-- NU_PARAM_B: double (nullable = true)
 |-- NU_PARAM_C: double (nullable = true)
 |-- TX_COR: string (nullable = true)
 |-- CO_PROVA: integer (nullable = true)
 |-- TP_LINGUA: integer (nullable = true)
 |-- IN_ITEM_ADAPTADO: integer (nullable = true)
 |-- TP_VERSAO_DIGITAL: integer (nullable = true)



In [ ]:
df_itens_prova.write.mode('overwrite').parquet(itens_prova_silver_write_path)

In [ ]:
df_microdados.write.mode('overwrite').parquet(microdados_silver_write_path)

In [ ]:
itens_prova_gold_write_path = 's3a://enem-pyspark-athena-gold/itens_prova/'
microdados_gold_write_path = 's3a://enem-pyspark-athena-gold/microdados/'

In [15]:
df_itens_prova = df_itens_prova.withColumn('NU_ANO', lit(2020))

In [16]:
df_itens_prova.show()

+----------+-------+-------+-----------+-------------+------------+--------------+----------+----------+----------+----------------+--------+---------+----------------+-----------------+------+
|CO_POSICAO|SG_AREA|CO_ITEM|TX_GABARITO|CO_HABILIDADE|IN_ITEM_ABAN|TX_MOTIVO_ABAN|NU_PARAM_A|NU_PARAM_B|NU_PARAM_C|          TX_COR|CO_PROVA|TP_LINGUA|IN_ITEM_ADAPTADO|TP_VERSAO_DIGITAL|NU_ANO|
+----------+-------+-------+-----------+-------------+------------+--------------+----------+----------+----------+----------------+--------+---------+----------------+-----------------+------+
|        26|     LC|   7099|          D|           15|           0|          null|   1.33337|  -0.37357|   0.00658|LARANJA - BRAILE|     623|     null|               1|             null|  2020|
|        43|     LC|   7099|          D|           15|           0|          null|   1.33337|  -0.37357|   0.00658|          BRANCA|     620|     null|               0|             null|  2020|
|        26|     LC|   7099|  

In [ ]:
df_itens_prova.write.partitionBy('NU_ANO').mode('overwrite').parquet(itens_prova_gold_write_path)

In [ ]:
df_microdados.write.partitionBy('NU_ANO').mode('overwrite').parquet(microdados_gold_write_path)

In [23]:
df_microdados.limit(10).toPandas()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,NaN,2020,11,F,1,2,1,1,11,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,1501402,Belém,15,PA,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,1,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,NaN,2020,11,M,2,3,1,1,11,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,2408102,Natal,24,RN,1,1,1,1,702.0,689.0,693.0,698.0,NaN,NaN,NaN,NaN,BCBDBDCCCDBDDBADEEACEEAECBECCDBCACDEBEECEBAAE,BCAECABCDCEBDBBBDAABABACBCDDDBDDBABDAADACACBC,99999CADDEDADBAABEAADAAAADCADAABDCCBEECAEECAAC...,EBEBDEDAECBADCADDABACAEDBEEEAEADCCBCEACDACEBB,1,ABBACBCCCDDDDBAEEBECEDACABDCBEDAECDABECCBBADE,BCEECDBCCDEBDBBBBEAAAEACBCDDDBDDBABDABDDBAABC,99999CBDDEDBDBACECABAAAEAECCDCABCBBBBEEDCECAAE...,BBEADECAECBBXCEBADBACAEDACEDADAADCBCCBBDABBEE,1.0,120.0,120.0,120.0,120.0,100.0,580.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,NaN,2020,4,F,2,3,2,2,0,2,1.0,0,2927408.0,Salvador,29.0,BA,2.0,1.0,1.0,2927408,Salvador,29,BA,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,0,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,C,A,D,3,B,A,B,A,A,A,B,A,B,A,B,A,A,B,A,A,A,A,A,A
3,NaN,2020,2,M,1,3,1,2,0,2,1.0,0,3547304.0,Santana de Parnaíba,35.0,SP,3.0,1.0,1.0,3547304,Santana de Parnaíba,35,SP,1,1,1,1,700.0,688.0,692.0,696.0,NaN,675.0,NaN,NaN,EBEDCCCDCBDBAECAECCECEACCBAABBBCEABCAAAEBEBAE,DABCCACCBCCDCADBDCBAAEBDDBAABDBAEBEBBABDABDBE,DCEAB99999AADAECCEBCDDDCBABDDABBCEEBAEDDDADEBE...,CBDBDCCDDEECBAABBBADEDEDBECDEBBADAEBAABBECBBA,0,BDECCACBEBDEAEDAECCBAEABCCEADDBBCADCBDDEBDBAC,DABCCAECBABECADBDCCABDBCDDAABDBAEBEBBDBDDBDBE,DCEAD99999AADACCCCBBDDCABAECAABBEECBAEDDBBEEBE...,EBDBXCCDAEECBAABABAEDDEDAECBCDCADEEBBABBACBCA,1.0,140.0,200.0,140.0,120.0,160.0,760.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,NaN,2020,4,F,1,3,2,1,1,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,3121605,Diamantina,31,MG,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,1,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,G,B,B,3,B,A,B,D,A,A,B,A,B,A,A,A,A,B,A,B,B,A,A,B
5,NaN,2020,3,F,1,3,1,1,1,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,4305207,Cerro Largo,43,RS,1,1,1,1,598.0,567.0,577.0,588.0,NaN,NaN,NaN,NaN,CABBEEACAEBEBCCDACBEDADDCCBECDCBDADDEBDBDACAB,ACEBBDDCADDAACEEAAEBDBDCCDACCDEDCDACADBBBECDD,99999CBECEBEDBEECEABBADCDDADCADAAECDEBAAADEBAD...,BADCAAAEEDBEDBACEDBACCDBABDECCDDCCAEAACEDACCB,1,CEDBDDDCACCBDAEBADCBDCDXAAECEDDDECDDCBDDAEDCC,DABCBDDDAEDBECECBBEDEBDEECBCCCBACDCCADEBDECDD,AAEDCACEEECEAACBAEBBEDACDBAADADDEBBEBAAADEACAA...,AEBCCXCCCACCDABCCABCABCBAECEDBBDBEEEDBBACCDAC,1.0,140.0,120.0,140.0,140.0,160.0,700.0,C,B,C,B,5,C,A,B,E,A,A,B,A,A,A,B,A,A,B,B,A,D,A,B,B
6,NaN,2020,9,M,1,3,1,1,6,1,NaN,0,NaN,None,NaN,None,NaN,NaN,NaN,2611606,Recife,26,PE,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,0,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,NaN,2020,4,M,1,1,1,1,

Sobre o ENEM 2020, qual é a média da nota em matemática apenas para as alunas do sexo Feminino?

In [28]:
df_microdados.filter((df_microdados['TP_SEXO'] == 'F'))\
                .select(mean('NU_NOTA_MT'))\
                .show()


+-----------------+
|  avg(NU_NOTA_MT)|
+-----------------+
|499.2338038965187|
+-----------------+



Qual é a média da nota em Ciências Humanas para os alunos do sexo masculino que estudaram numa escola no estado de São Paulo?

In [29]:
df_microdados.filter(df_microdados['TP_SEXO'] == 'M')\
                .filter(df_microdados['SG_UF_ESC'] == 'SP')\
                .select(mean('NU_NOTA_CH'))\
                .show()


+-----------------+
|  avg(NU_NOTA_CH)|
+-----------------+
|547.6993710691823|
+-----------------+



Qual é a média da nota em Ciências Humanas dos alunos que estudaram numa escola na cidade de Natal?

In [30]:
df_microdados.filter(df_microdados['NO_MUNICIPIO_ESC'] == 'Natal')\
                .select(mean('NU_NOTA_CH'))\
                .show()

+-----------------+
|  avg(NU_NOTA_CH)|
+-----------------+
|544.5664160401003|
+-----------------+



Qual é o município (município da escola) que obteve a maior nota MÉDIA em matemática?

In [43]:
df_microdados.groupBy('NO_MUNICIPIO_PROVA')\
    .agg(mean('NU_NOTA_MT').alias('media_mt'))\
    .sort('media_mt', ascending=False)\
    .toPandas()

,NO_MUNICIPIO_PROVA,media_mt
0,Valinhos,627.775862
1,São João da Boa Vista,609.868132
2,São José dos Campos,607.911408
3,Horizontina,604.312500
4,Botucatu,604.209459
5,Espírito Santo do Pinhal,603.666667
6,Lajeado,601.094595
7,São Caetano do Sul,599.746398
8,Garibaldi,598.961538
9,Cândido Mota,598.450000
